<a href="https://colab.research.google.com/github/DHNGUYEN68/TestGithub/blob/master/AdaBoost_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AdaBoost demo
Source : https://www.hdm-stuttgart.de/~maucher/Python/ComputerVision/html/Adaboost.html
AdaBoost (Adaptive Boosting) is a Meta Machine Learning algorithm. It defines a procedure to learn and combine a set of weak classifiers to form a strong classifier. The AdaBoost learning procedure consist of M iterations. In each iteration a weak classifier is learned using an appropriate Machine Learning algorithm. In each iteration the training samples that are classified correctly obtain a smaller weight, whereas the weight of all trainingsamples that are not correctly classified remain the same. The effect is that in the training procedure of the next iteration, the training samples that have been classified correctly, have less influence than the not correctly classified training samples. Thus the model in the next iteration is more adapted to the previously misclassified samples. The final classifier is a linear combination of all M weak classifiers. The coefficients in this linear combination are also determined in the iterations: Weak classifiers with a small error rate obtain a larger weight, than weak classifiers with a high error rate.

AdaBoost is applied e.g. in the Viola-Jones face detector. Here, in each iteration a weak classifier is learned that just applies a threshold on a single feature. The learned weak classifier is the single feature with a corresponding threshold, that best discriminates positive (face subwindows) from negative samples (non-face subwindows). Note that this optimal single feature and the corresponding threshold depends on the varying weights of the training samples. AdaBoost in Viola-Jones face detection thus not only learns a classifier but also the most informative features out of a very large set of features

In [0]:
import numpy as np
from sklearn import linear_model
from sklearn import metrics


X=np.array([[3,5,2],
            [5,5,5],
            [3,0,2],
            [3,4,1],
            [5,0,2],
            [2,1,3],
            [4,3,1],
            [2,5,1],
            [0,2,1],
            [2,1,5]])

print (X.shape)

(10, 3)


In the following example AdaBoost is applied to a set of 10 training samples, which are separated into 2 classes. Each sample is described by 3 features. The training samples are defined in matrix X, the corresponding class labels are defined in the vector C. Inititally all training samples obtain the same weight w=1/10. The weights are stored in variable W. The number of iterations (i.e. the number of weak classifiers) is defined to be M=5.:

In [0]:
C=np.array([0,0,0,0,0,1,1,1,1,1]) # class labels
print (C.shape)
print (C)

(10,)
[0 0 0 0 0 1 1 1 1 1]


In [0]:
C_bis = np.atleast_2d(C)
print (C_bis.shape)
print (C_bis)

(1, 10)
[[0 0 0 0 0 1 1 1 1 1]]


In [0]:
# concatenate the labels into the training data
print ("Example data - 3 features, last column is label")
print (np.hstack((X,np.transpose(np.atleast_2d(C)))))

Example data - 3 features, last column is label
[[3 5 2 0]
 [5 5 5 0]
 [3 0 2 0]
 [3 4 1 0]
 [5 0 2 0]
 [2 1 3 1]
 [4 3 1 1]
 [2 5 1 1]
 [0 2 1 1]
 [2 1 5 1]]


In [0]:
W=1.0/len(C)*np.ones(C.shape[0])
print ("Initial weights")
print (W)
M=5 #Number of iterations

Initial weights
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]


In each iteration of the following loop a weak classifier is learned. This is achieved by determining for each of the three features the threshold, that best separates the negative (class label =0) from the positive (class label = 1) training samples. The learning algorithm applied for this is the LogisticRegression module form scikit-learn. 

Once the best feature and corresponding threshold of the current iteration is determined, the weighted error J on the training data is applied to compute the variables beta and alpha. Variable beta is applied to adapt the weights of the training samples for the next iteration. 

Variable alpha (voting power)is the coefficient that determines how strong the currently learned weak classifier contributes to the final classifier.:

In [0]:
# From sci-kit learn
reg=linear_model.LogisticRegression()

# Array to store 
# 1.)best feature, 
# 2.)threshold, 
# 3.)alpha of best classifier in each iteration
bestClassifier=np.zeros((M,3)) 

for F in range(M):
        print ("\n\n")
        print ("#"*30)
        print ("Start of Iteration %d"%F)
        IndicatorList=[]
        betaList=[]
        thresholdList=[]
        bestVal=10
        bestFeat=10
        for i in range(X.shape[1]):
                print ("\nClassification with respect to feature %d"%i)
                indata= np.transpose(np.atleast_2d(X[:,i]))
                reg.fit(indata,C)
                threshold=-1*reg.intercept_/reg.coef_
                print ("Threshold = ",threshold)
                CE=reg.predict(indata)
                print ("target :     ",C)
                print ("predicted :  ",CE)
                Indicator = np.abs(CE-C)
                errors=np.dot(Indicator,W)
                print ("J =",errors)
                if errors < bestVal:
                        bestVal=errors
                        bestFeat=i
                J=errors
                beta=1.0*J/(1-J)
                IndicatorList.append(Indicator)
                betaList.append(beta)
                thresholdList.append(threshold)
                print ('-'*30)

        I=IndicatorList[bestFeat]
        beta= betaList[bestFeat]
        threshold=thresholdList[bestFeat]
        alpha=np.log(1.0/beta)
        print ("BestFeature:          ",bestFeat)
        print ("Threshold =           ",threshold)
        print ("Best Beta =           ",beta)
        print ("Corresponding alpha = ",alpha)
        bestClassifier[F,:]=np.array([bestFeat,threshold,alpha])
        for r in range(C.shape[0]):
                W[r]=W[r]*np.power(beta,1-I[r])
        Wnorm=np.sum(W)
        W=W/Wnorm
        print ("New weights for training samples: ")
        print (W)
        print ("#"*30)
        print ("\n\n")




##############################
Start of Iteration 0

Classification with respect to feature 0
Threshold =  [[2.02868822]]
target :      [0 0 0 0 0 1 1 1 1 1]
predicted :   [0 0 0 0 0 1 0 1 1 1]
J = 0.1
------------------------------

Classification with respect to feature 1
Threshold =  [[1.85487592]]
target :      [0 0 0 0 0 1 1 1 1 1]
predicted :   [0 0 1 0 1 1 0 0 0 1]
J = 0.5
------------------------------

Classification with respect to feature 2
Threshold =  [[1.64177335]]
target :      [0 0 0 0 0 1 1 1 1 1]
predicted :   [0 0 0 1 0 0 1 1 1 0]
J = 0.30000000000000004
------------------------------
BestFeature:           0
Threshold =            [[2.02868822]]
Best Beta =            0.11111111111111112
Corresponding alpha =  2.1972245773362196
New weights for training samples: 
[0.05555556 0.05555556 0.05555556 0.05555556 0.05555556 0.05555556
 0.5        0.05555556 0.05555556 0.05555556]
##############################






##############################
Start of Iteration 1



At this stage all M=5 weak classifiers are learned. The parameters feature index, threshold and alpha of each weak classifier are stored in the numpy-array bestClassifiers. These parameters uniquelly define the final strong classifier. The strong classifier is finally applied to three different input vectors, defined in the array newdata. The decision criteria is: If the linear combination of the strong classifiers is larger than half of the sum over all 5 alpha-values, then the predicted class label is 0, otherwise it is 1.:

In [0]:
import math

print ("*"*50)
print ("Best Classifiers in each iteration:")
print ("Feature|Threshold|Alpha")
print (bestClassifier)
halfSumAlpha=0.5*np.sum(bestClassifier[:,2])
print ("Threshold of combined Classifier  :  ",halfSumAlpha)
newdata=np.array([[1,5,1],[2,10,2],[5,3,1]])
for u in range(newdata.shape[0]):
        data=newdata[u,:]
        sum=0
        for v in range(M):
                if newdata[u,math.floor(bestClassifier[v,0])] > bestClassifier[v,1]:
                        sum=sum+bestClassifier[v,2]
        print ("Data : ",newdata[u,:])
        if sum > halfSumAlpha:
                print ("Classified as Class 0")
        else:
                print ("Classified as Class 1")

**************************************************
Best Classifiers in each iteration:
Feature|Threshold|Alpha
[[0.         2.02868822 2.19722458]
 [2.         1.64177335 1.60943791]
 [0.         2.02868822 0.84729786]
 [2.         1.64177335 0.58778666]
 [0.         2.02868822 0.45198512]]
Threshold of combined Classifier  :   2.84686606940135
Data :  [1 5 1]
Classified as Class 1
Data :  [ 2 10  2]
Classified as Class 1
Data :  [5 3 1]
Classified as Class 0
